# mT5 - In Progress

In [1]:
# @title Pips
!pip install transformers==4.0.0rc1 -q
!pip install transformers==4.0.0rc1 sentencepiece -q

     |████████████████████████████████| 1.4MB 9.2MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
     |████████████████████████████████| 2.9MB 43.5MB/s 
     |████████████████████████████████| 1.1MB 8.1MB/s 


In [2]:
# @title Imports

from transformers import MT5ForConditionalGeneration, AutoTokenizer
import torch
model = MT5ForConditionalGeneration.from_pretrained("seduerr/mt5-paraphrases-espanol")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
# @title Inference
from transformers import MT5ForConditionalGeneration, AutoTokenizer
import torch
model = MT5ForConditionalGeneration.from_pretrained("seduerr/mt5-paraphrases-espanol")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

sentence = "El pacto implica el establecimiento por parte de Rabat de relaciones diplomáticas con Israel."  #@param {type: "string"}
text =  "paraphrase: " + sentence + " </s>"
max_len = 256
encoding = tokenizer.encode_plus(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

beam_outputs = model.generate(
    input_ids=input_ids, 
    attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=50,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=10
)

print ("Paraphrased Phrase: ")
final_outputs = []
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

Paraphrased Phrase: 
0: El pacto implica el establecimiento por parte de Rabat de relaciones diplomáticas.
1: El pacto implica la establecimiento por parte de Rabat de relaciones diplomáticas con Israel.
2: El pacto implica el establecimiento por parte de Rabat por relaciones diplomáticas con Israel.
3: El pacto implica el establecimiento de relaciones diplomáticas con Israel.
